In [7]:
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import subprocess
import pybedtools

In [52]:
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
PWD = Path().cwd()
#MAZUR = Path("/data/mazurovev/all_marks/methylation/lncRNA_Peaks_corrs").resolve()
DATA = Path("../data")
METHYL = DATA / "methylation_peaks"
METHYL_CONCAT = DATA / "methylation_peaks_concat"
ANNO = Path("/export/home/public/agletdinov_shared/annotations")
PREPROCESSING = Path("../preprocessing")
METHYL_INDEX = PREPROCESSING / "methyl_index"
METHYL_BED = PREPROCESSING / "methyl_bed"

In [3]:
import gzip
import pandas as pd

# Функция для чтения GTF-файла и подсчета уникальных генов
def count_genes_in_gtf(gtf_file):
    # Открываем файл
    with gzip.open(gtf_file, 'rt') as f:
        # Считываем строки, относящиеся к генам
        gene_lines = [line for line in f if 'gene' in line and line.split('\t')[2] == 'gene']
        
    # Выделяем ID генов из строк
    gene_ids = set()
    for line in gene_lines:
        # Разбиваем строку на колонки
        columns = line.split('\t')
        # Последняя колонка содержит атрибуты, ищем gene_id
        attributes = columns[8]
        for attribute in attributes.split(';'):
            if 'gene_id' in attribute:
                gene_id = attribute.split('"')[1]
                gene_ids.add(gene_id)
                break
    
    return len(gene_ids)

In [4]:
def get_all_correlated_peaks(hm, to_hdf5, to_save):
    if to_save.exists():
        print(f"Skipped: {to_hdf5.stem}")
        return None
    all_results = []

    # Открываем HDF5 файл
    with h5py.File(to_hdf5, 'r') as f:
        lncRNAs = list(f['lncRNAs_names'][:])
        corrs_matrix = f['corrs_matrix'][:]

    # Читаем пики
    peaks = pd.read_csv(Path("/data/mazurovev/all_marks") / hm / "merged_peaks_first_in_biosample.bed", sep="\t", header=None)

    for i, lncRNA in enumerate(lncRNAs):
        corrs = corrs_matrix[i, :]
        nonzero_indices = np.nonzero(corrs)[0]
        nonzero_corrs = corrs[nonzero_indices]
        nonzero_peaks = ["peak_" + str(j) for j in nonzero_indices]

        # Извлекаем соответствующие пики
        res = peaks[peaks[3].isin(nonzero_peaks)].copy()
        res["corr"] = nonzero_corrs
        res["lnc_ens"] = lncRNA.decode('utf-8')  # Декодируем байтовую строку в строку

        all_results.append(res)

    # Объединяем все результаты в один DataFrame
    final_result = pd.concat(all_results, ignore_index=True)
    
    final_result.to_csv(to_save, sep="\t", compression="gzip", index=False)

for to_hdf5 in MAZUR.glob("*.hdf5"):
    print(to_hdf5.stem)
    to_save = METHYL / f"{to_hdf5.stem}.tsv.gz"
    get_all_correlated_peaks(hm="methylation", to_hdf5=to_hdf5, to_save=to_save)

In [5]:
def indexing_methyl(chunk, lnc_ens, corr_sign, subdf):
    to_index = METHYL_INDEX / chunk
    to_index.mkdir(exist_ok=True, parents=True)
    to_save = to_index / f"methyl-{lnc_ens}-{corr_sign}-.tsv.gz"
    if to_save.exists():
        print(f"Skipped: methyl {lnc_ens} {corr_sign}")
        return None
    subdf.to_csv(to_save, sep="\t", compression="gzip", index=False)

In [8]:
for to_df in METHYL.glob("*.tsv.gz"):
    print(to_df.stem)
    chunk = to_df.stem.split("_")[6]
    df = pd.read_csv(to_df, sep="\t")
    df.rename(columns={"0":"chrom", "1":"start", "2":"end", "3":"name"}, inplace=True)
    df["corr"] = df["corr"].apply(lambda x: np.round(x, 3))
    df["corr_sign"] = np.where(df["corr"] > 0, "plus", "minus")
    for (lnc_ens, corr_sign), subdf in df.groupby(by=["lnc_ens", "corr_sign"]):
        indexing_methyl(chunk=chunk, lnc_ens=lnc_ens, corr_sign=corr_sign, subdf=subdf)

lncRNA_Peaks_Correlations_corrected_non_zero_0.tsv



KeyboardInterrupt



In [ ]:
pd.concat([pd.read_csv(to_df, sep="\t", dtype=dtype, header=None).loc[1:] for to_df in METHYL_INDEX.glob(f"*{sign}*")])

In [10]:
#Не хватает оперативной памяти
dtype = {
    "chrom": str,
    "start": np.uint32,
    "end": np.uint32
}
all_peaks = {}
for sign in "minus","plus":
    all_peaks[sign] = pd.concat([pd.read_csv(to_df, sep="\t", dtype=dtype, header=None).loc[1:] for to_df in METHYL_INDEX.glob(f"*{sign}*")]).reset_index(drop=True)\
    .drop_duplicates().dropna()
    
all_peaks_df = pd.concat([all_peaks[sign] for sign in ("minus","plus")]).drop_duplicates()
del all_peaks

/tmp/ipykernel_56201/652964208.py:9: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_peaks[sign] = pd.concat([pd.read_csv(to_df, sep="\t", dtype=dtype, header=None).loc[1:] for to_df in METHYL_INDEX.glob(f"*{sign}*")]).reset_index(drop=True)\
/tmp/ipykernel_56201/652964208.py:9: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_peaks[sign] = pd.concat([pd.read_csv(to_df, sep="\t", dtype=dtype, header=None).loc[1:] for to_df in METHYL_INDEX.glob(f"*{sign}*")]).reset_index(drop=True)\
/tmp/ipykernel_56201/652964208.py:9: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_peaks[sign] = pd.concat([pd.read_csv(to_df, sep="\t", dtype=dtype, header=None).loc[1:] for to_df in METHYL_INDEX.glob(f"*{sign}*")]).reset_index(drop=True)\
/tmp/ipykernel_56201/652964208.py:9: DtypeWarning: Columns (1,2,4) have mixed types. 

KeyboardInterrupt: 

In [12]:
ENSG00000280047 = pd.concat([pd.read_csv(to_df, sep="\t", header=None).loc[1:] for to_df in METHYL_INDEX.glob("*ENSG00000280047*minus*")]).reset_index(drop=True)
ENSG00000280047

,0,1,2,3,4,5,6
0,chr13,99062576,99062577,peak_13294105,-0.855,ENSG00000280047,minus
1,chr13,99062606,99062607,peak_13294107,-0.89,ENSG00000280047,minus
2,chr13,99062612,99062613,peak_13294109,-0.856,ENSG00000280047,minus
3,chr2,32758365,32758366,peak_25313521,-0.869,ENSG00000280047,minus


In [75]:
all_methil_merged = pd.concat([pd.read_csv(to_df, sep="\t", header=None).loc[1:] for to_df in METHYL_INDEX.glob("*")]).reset_index(drop=True)

/tmp/ipykernel_73202/2367621633.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_methil_merged = pd.concat([pd.read_csv(to_df, sep="\t", header=None).loc[1:] for to_df in METHYL_INDEX.glob("*")]).reset_index(drop=True)
/tmp/ipykernel_73202/2367621633.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_methil_merged = pd.concat([pd.read_csv(to_df, sep="\t", header=None).loc[1:] for to_df in METHYL_INDEX.glob("*")]).reset_index(drop=True)
/tmp/ipykernel_73202/2367621633.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_methil_merged = pd.concat([pd.read_csv(to_df, sep="\t", header=None).loc[1:] for to_df in METHYL_INDEX.glob("*")]).reset_index(drop=True)
/tmp/ipykernel_73202/2367621633.py:1: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.


KeyboardInterrupt: 

In [17]:
to_save = METHYL_CONCAT / "all_methil_merged.tsv.gz"
all_methil_merged.to_csv(to_save, sep="\t", compression="gzip", index=False)


KeyboardInterrupt



In [5]:
def tsv_to_bed(to_tsv):
    to_save = METHYL_BED / f"{to_tsv.name.split('.')[0]}.bed"
    if to_save.exists():
        print(f"Skipped: {to_save}")
        return None
    to_save.parent.mkdir(exist_ok=True, parents=True)
    df = pd.read_csv(to_tsv, sep="\t")
    stream = open(to_save, 'w')
    for i in range(len(df)):
        stream.write(f"{df.loc[i, 'chrom']}\t{df.loc[i, 'start']}\t{df.loc[i, 'end']}\n")
    
    stream.close()
    #cmd = f"gzip {to_save}" 
    #subprocess.run(cmd, shell=True, check=True)

In [6]:
with Parallel(n_jobs=-1) as pool:
    chunks = pool(
        delayed(tsv_to_bed)(to_tsv)
        for to_tsv in METHYL_INDEX.glob("*")
    )

In [21]:
test_1 = pybedtools.BedTool(METHYL_BED / 'methyl-ENSG00000099869-minus-.bed')
test_1 = test_1.sort()
test_2 = pybedtools.BedTool(METHYL_BED / 'methyl-ENSG00000103472-minus-.bed')
test_2 = test_2.sort()

In [25]:
print(test_1)

chr12	49123587	49123588



In [26]:
print(test_2)

chr1	15837335	15837336
chr16	19885468	19885469
chr19	36064797	36064798



In [23]:
print(test_1.cat(test_2))

chr1	15837335	15837336
chr12	49123587	49123588
chr16	19885468	19885469
chr19	36064797	36064798



In [27]:
files = [i for i in METHYL_BED.glob('*.bed')]
x = pybedtools.BedTool(files[0])
bed_concat = x.cat(*files[1:])

In [32]:
len(bed_concat)

17758261

In [34]:
bed_concat = bed_concat.sort()

BEDToolsError: 
Command was:

	bedtools sort -i /tmp/pybedtools.ei4hweqq.tmp

Error message was:
Error: The requested file (/tmp/pybedtools.ei4hweqq.tmp) could not be opened. Error message: (No such file or directory). Exiting!


In [33]:
bed_concat.moveto(METHYL_BED / "methyl_concat" /'methyl_concat.bed')

<BedTool(../preprocessing/methyl_bed/methyl_concat/methyl_concat.bed)>

In [40]:
peaks_anno_df = pd.read_csv(METHYL_BED / "methyl_concat/peaks_anno.csv", sep="\t")

In [41]:
peaks_anno_df.head()

,seqnames,start,end,width,strand,peak,feature,start_position,end_position,feature_strand,insideFeature,distancetoFeature,shortestDistance,fromOverlappingOrNearest
1,chr1,10542,10542,1,*,X00000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,10571,10571,1,*,X00000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,10577,10577,1,*,X00000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,10579,10579,1,*,X00000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,chr1,10610,10610,1,*,X00000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
a = peaks_anno_df["feature"].unique()

In [44]:
a[:10]

array([nan, 'ENSG00000225880.4', 'ENSG00000240453.1', 'CATG00000042729.1',
       'CATG00000042732.1', 'CATG00000042734.1', 'ENSG00000228327.2',
       'ENSG00000237491.4', 'ENSG00000177757.1', 'ENSG00000228794.4'],
      dtype=object)

In [45]:
peaks_anno_df[peaks_anno_df["feature"] == 'ENSG00000225880.4']

,seqnames,start,end,width,strand,peak,feature,start_position,end_position,feature_strand,insideFeature,distancetoFeature,shortestDistance,fromOverlappingOrNearest
182,chr1,91684,91685,2,*,X00000182,ENSG00000225880.4,91421.0,762886.0,-,inside,671202.0,263.0,Overlapping
184,chr1,92392,92392,1,*,X00000183,ENSG00000225880.4,91421.0,762886.0,-,inside,670494.0,971.0,Overlapping
186,chr1,92742,92742,1,*,X00000184,ENSG00000225880.4,91421.0,762886.0,-,inside,670144.0,1321.0,Overlapping
188,chr1,92763,92763,1,*,X00000185,ENSG00000225880.4,91421.0,762886.0,-,inside,670123.0,1342.0,Overlapping
190,chr1,93393,93393,1,*,X00000186,ENSG00000225880.4,91421.0,762886.0,-,inside,669493.0,1972.0,Overlapping
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3304,chr1,759226,759226,1,*,X00001351,ENSG00000225880.4,91421.0,762886.0,-,inside,3660.0,3660.0,Overlapping
3307,chr1,759318,759318,1,*,X00001352,ENSG00000225880.4,91421.0,762886.0,-,inside,3568.0,3568.0,Overlapping
3310,chr1,759876,759876,1,*,X00001353,ENSG00000225880.4,91421.0,762886.0,-,inside,3010.0,3010.0,Overlapping
3313,chr1,761800,761801,2,*,X00001354,ENSG00000225880.4,91421.0,762886.0,-,inside,1086.0,1085.0,Overlapping


In [46]:
759608 - 91421

668187

In [47]:
762886 - 762919

-33

In [ ]:
762919

In [ ]:
91421.0	762886.0	

In [78]:
to_csv= METHYL_CONCAT / "all_methil_merged.tsv.gz"
all_methil_merged = pd.read_csv(to_csv, sep="\t", compression="gzip")

In [98]:
all_methil_merged

,0,1,2,3,4,5,6
0,chr11,65540025,65540026,peak_8221299,0.824,ENSG00000274317,plus
1,chr16,88534857,88534858,peak_18894879,0.852,ENSG00000274317,plus
2,chr17,74861803,74861804,peak_20845738,0.804,ENSG00000274317,plus
3,chr19,4911993,4911994,peak_22893683,0.831,ENSG00000274317,plus
4,chr21,35046939,35046940,peak_30600780,0.867,ENSG00000274317,plus
...,...,...,...,...,...,...,...
277825392,chr7,71152278,71152279,peak_45219202,-0.594,ENSG00000273080,minus
277825393,chr7,71152353,71152354,peak_45219204,-0.636,ENSG00000273080,minus
277825394,chr7,71152851,71152852,peak_45219214,-0.573,ENSG00000273080,minus
277825395,chr7,71152920,71152921,peak_45219216,-0.600,ENSG00000273080,minus


In [104]:
all_methil_merged_filt = all_methil_merged.drop_duplicates(subset=["0","1","2","5"])
all_methil_merged_filt.shape

(277825397, 7)

In [105]:
all_methil_merged_filt = all_methil_merged.drop_duplicates(subset=["0","1","2"])
all_methil_merged_filt.shape

(25177317, 7)

In [100]:
mask = all_methil_merged["5"].apply(lambda x: "ENSG00000225880" in x)
all_methil_merged[mask]

,0,1,2,3,4,5,6


In [79]:
all_methil_merged.shape

(277825397, 7)

In [102]:
all_methil_merged.loc[277825392]

0               chr7
1           71152278
2           71152279
3      peak_45219202
4             -0.594
5    ENSG00000273080
6              minus
Name: 277825392, dtype: object

In [108]:
all_methil_merged[all_methil_merged["1"] == 10541]

,0,1,2,3,4,5,6
249128659,chr4,10541,10542,peak_35232603,-0.543,ENSG00000278989,minus


In [103]:
all_methil_merged[all_methil_merged["3"] == "peak_45219202"]

,0,1,2,3,4,5,6
3484824,chr7,71152278,71152279,peak_45219202,0.634,ENSG00000267506,plus
6831216,chr7,71152278,71152279,peak_45219202,0.642,ENSG00000248441,plus
8359489,chr7,71152278,71152279,peak_45219202,0.657,ENSG00000227888,plus
12861960,chr7,71152278,71152279,peak_45219202,0.742,ENSG00000258498,plus
15595068,chr7,71152278,71152279,peak_45219202,-0.619,ENSG00000254614,minus
23519914,chr7,71152278,71152279,peak_45219202,0.616,ENSG00000256576,plus
26166893,chr7,71152278,71152279,peak_45219202,-0.686,ENSG00000257913,minus
35300918,chr7,71152278,71152279,peak_45219202,-0.611,ENSG00000228649,minus
40754533,chr7,71152278,71152279,peak_45219202,0.564,ENSG00000280234,plus
48872330,chr7,71152278,71152279,peak_45219202,0.667,ENSG00000267272,plus


In [80]:
peaks_anno_df.shape

(25200691, 14)

In [82]:
bed_concat = pybedtools.BedTool(METHYL_BED / "methyl_concat" /'methyl_concat.bed')

In [83]:
len(bed_concat)

17758261

In [ ]:
methyl_peaks = pd.merge(left=all_methil_merged, left_on="target.gapmerID",
         right=fibro_aso[["aso_id", "target.geneID"]], right_on="aso_id")
fibro_expr_filt.head()

In [10]:
!ls /data/mazurovev/all_marks/methylation/lncRNA_Peaks_corrs

lncRNA_Peaks_Correlations_corrected_non_zero_0.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_10.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_11.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_12.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_13.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_14.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_15.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_16.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_17.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_18.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_1.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_2.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_3.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_4.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_5.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_6.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_7.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_8.hdf5
lncRNA_Peaks_Correlations_corrected_non_zero_9.hdf5


In [11]:
!ls {METHYL}

lncRNA_Peaks_Correlations_corrected_non_zero_0.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_10.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_11.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_12.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_13.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_14.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_15.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_16.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_17.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_18.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_1.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_2.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_3.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_4.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_5.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_6.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_7.tsv.gz
lncRNA_Peaks_Correlations_corrected_non_zero_8.tsv.gz
lncRNA_Peaks_Correl

In [ ]:
corr_df = pd.concat([pd.read_csv(to_df, sep="\t") for to_df in METHYL.glob("*.tsv.gz")])
corr_df.sort_values(by=["lnc_ens", "0", "1"], inplace=True)
corr_df.shape

In [64]:
# Указываем путь к файлу
gtf_file = ANNO / "FANTOM_CAT.lv2_permissive.gtf.gz"
anno = pd.read_csv(gtf_file, sep="\t", header=None)
anno.shape

(5181581, 9)

In [72]:
mask = anno[8].apply(lambda x: "ENSG00000225880" in x)

In [73]:
anno[mask]

,0,1,2,3,4,5,6,7,8
4778223,chr1,FANTOM,gene,91421,762886,.,-,.,"gene_id ""ENSG00000225880.4""; geneSuperClass ""a..."
4778224,chr1,FANTOM,transcript,761586,762886,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""EN..."
4778225,chr1,FANTOM,exon,761586,762886,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""EN..."
4778226,chr1,FANTOM,transcript,759426,762598,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""EN..."
4778227,chr1,FANTOM,exon,759426,762598,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""EN..."
4778228,chr1,FANTOM,transcript,91421,762886,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""FT..."
4778229,chr1,FANTOM,exon,91421,91629,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""FT..."
4778230,chr1,FANTOM,exon,92091,92240,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""FT..."
4778231,chr1,FANTOM,exon,259017,259121,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""FT..."
4778232,chr1,FANTOM,exon,655412,655580,.,-,.,"gene_id ""ENSG00000225880.4""; transcript_id ""FT..."
